In [ ]:
from gensim.models import KeyedVectors

In [ ]:
w2v=KeyedVectors.load("/content/drive/MyDrive/comparative_Analysis_of_embeddings/Sentiment140/Word2Vec/w2c")

In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/drive/MyDrive/comparative_Analysis_of_embeddings/Sentiment140/sentiment140-subset-preprocessed.csv")
y=df["polarity"]


In [ ]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

65


In [ ]:
import tensorflow as tf
maxlen=66
max_words=len(w2v.wv.vocab.keys())
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])
X

array([[    0,     0,     0, ...,    12,     9,    72],
       [    0,     0,     0, ...,   137,    14,  2628],
       [    0,     0,     0, ...,     7,   116,  4375],
       ...,
       [    0,     0,     0, ...,  4583,   318, 11391],
       [    0,     0,     0, ...,     0,  9807,   449],
       [    0,     0,     0, ...,     1,    52, 95570]], dtype=int32)

In [ ]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
for word,i in tokenizer.word_index.items():
  if word in w2v:
    embed_vector=w2v[word]
  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [ ]:
embed_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.72822678,  0.7488817 , -1.08043909, ...,  0.63756752,
         0.96165609, -0.1680413 ],
       [-1.78625202,  1.72100365, -1.29291737, ...,  0.59565586,
        -0.29386473,  1.10913849],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout,Embedding,GRU,Dense,Flatten
model=Sequential()
model.add(Embedding(input_dim=max_words,output_dim=300,input_length=66,embeddings_initializer=Constant(embed_matrix)))
model.add(Flatten()) # loss stucks at about 
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=1)

Epoch 1/100
1172/1172 [==============================] - 20s 12ms/step - loss: 0.6548 - accuracy: 0.5988
Epoch 2/100
1172/1172 [==============================] - 12s 10ms/step - loss: 0.5961 - accuracy: 0.6752
Epoch 3/100
1172/1172 [==============================] - 12s 10ms/step - loss: 0.5503 - accuracy: 0.7232
Epoch 4/100
1172/1172 [==============================] - 12s 10ms/step - loss: 0.5091 - accuracy: 0.7569
Epoch 5/100
1172/1172 [==============================] - 12s 10ms/step - loss: 0.4703 - accuracy: 0.7860
Epoch 6/100
1172/1172 [==============================] - 12s 10ms/step - loss: 0.4347 - accuracy: 0.8060
Epoch 7/100
1172/1172 [==============================] - 12s 10ms/step - loss: 0.3992 - accuracy: 0.8215
Epoch 8/100
1172/1172 [==============================] - 12s 10ms/step - loss: 0.3669 - accuracy: 0.8384
Epoch 9/100
1172/1172 [==============================] - 12s 10ms/step - loss: 0.3344 - accuracy: 0.8535
Epoch 10/100
1172/1172 [==============================]

In [ ]:
y_pred=model.predict(X_test)
for i in range(len(y_pred)):
  if y_pred[i]>0.5:
    y_pred[i]=1
  else:
    y_pred[i]=0

In [ ]:
#Metric calculation function
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))
display_metrics(y_test,y_pred)

0.73196
0.7319486050279793
0.732044440495619
0.7319865855272287
